# Postgress DB HW 1

In [47]:
# Импортируем библиотеку для работы с данными:
import pandas as pd

In [51]:
# Указываем путь в файлу:
path = '/Users/edwardgrey/Documents/GitHub/SF Data Bases and Math/customer_and_transaction.xlsx'

In [55]:
# Загружаем данные о транзакциях из Excel файла:
transaction_data = pd.read_excel(path, sheet_name='transaction')

# Загружаем данные о клиентах из того же Excel файла:
customer_data = pd.read_excel(path, sheet_name='customer')

# Объединяем данные о транзакциях и клиентах по общему столбцу 'customer_id':
transaction_and_customers = pd.merge(transaction_data, customer_data, on='customer_id')

In [56]:
# Выводим названия столбцов в виде DataFrame для наглялности:
pd.DataFrame(transaction_and_customers.columns, columns=['Column Name'])

,Column Name
0,transaction_id
1,product_id
2,customer_id
3,transaction_date
4,online_order
5,order_status
6,brand
7,product_line
8,product_class
9,product_size


In [90]:
# Создаем случайную выборку из данных для удобства дальнейшего использования:
data = transaction_and_customers.sample(10)

## Проектирование структуры реляционной базы данных в DBDiagram:

![DB Diagram](dbeaver.png)

## Разделение данных в соответствии с проектирование в DBDiagram:

In [92]:
customers_table = data[['customer_id', 
                        'first_name', 
                        'last_name', 
                        'gender', 
                        'DOB', 
                        'job_title', 
                        'job_industry_category',
                        'wealth_segment',
                        'deceased_indicator',
                        'owns_car',
                        'address', 
                        'postcode',
                        'state',
                        'country',
                        'property_valuation'
]]

In [93]:
transactions_table = data[['transaction_id',
                            'product_id',
                            'customer_id',
                            'transaction_date',
                            'online_order',
                            'order_status'
]]

In [94]:
products_table = data[['product_id',
                        'brand',
                        'product_line',
                        'product_class',
                        'product_size',
                        'list_price',
                        'standard_cost'  
]]

In [95]:
# Сохраняем DataFrames в файлы CSV для последующей загрузки в Postgres:
customers_table.to_csv('customers_table.csv', index=False)
transactions_table.to_csv('transactions_table.csv', index=False)
products_table.to_csv('products_table.csv', index=False)

## Создание Таблиц в DBeaver с Postgres SQL:

```sql
CREATE TABLE customers (
  customer_id INT PRIMARY KEY NOT NULL,
  first_name VARCHAR NOT NULL,
  last_name VARCHAR NOT NULL,
  gender VARCHAR,
  DOB DATE,
  job_title VARCHAR,
  job_industry_category VARCHAR,
  wealth_segment VARCHAR,
  deceased_indicator VARCHAR,
  owns_car VARCHAR,
  address VARCHAR,
  postcode INT,
  state VARCHAR,
  country VARCHAR,
  property_valuation INT
);

CREATE TABLE products (
  product_id INT PRIMARY KEY NOT NULL,
  brand VARCHAR,
  product_line VARCHAR,
  product_class VARCHAR,
  product_size VARCHAR,
  list_price DECIMAL,
  standard_cost DECIMAL
);

CREATE TABLE transactions (
  transaction_id INT PRIMARY KEY NOT NULL,
  product_id INT,
  customer_id INT,
  transaction_date DATE,
  online_order VARCHAR,
  order_status VARCHAR,
  FOREIGN KEY (product_id) REFERENCES Products(product_id),
  FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
);
```

##  Комментарии

Третья нормальная форма (3НФ) предполагает, что в таблице отсутствуют транзитивные зависимости, то есть неключевые атрибуты не зависят от других неключевых атрибутов.

1. Customers (Клиенты):

`customer_id` - первичный ключ, уникально идентифицирует каждого клиента.

Все остальные атрибуты зависят только от `customer_id` и не имеют транзитивных зависимостей.
Например, `postcode` зависит от `customer_id` и не зависит от других атрибутов.
Таким образом, таблица сustomers соответствует третьей нормальной форме.

2. Products (Продукты):

`product_id` - первичный ключ, уникально идентифицирует каждый продукт.

Замещающий ключ: В данном датасете `product_id` не является уникальным и имеет дубликаты, возможно рассмотреть использование замещающего или суррогатного ключа:

```sql
product SERIAL PRIMARY KEY,
```

В этом случае база данных автоматически сгенерирует уникальный `product_id` для каждого нового продукта. Поскольку в данном решении я использую выборку без дубликатов, я не прибегаю к генерации суррогатного и композитного ключей.

Все остальные атрибуты зависят только от `product_id` и не имеют транзитивных зависимостей. Таким образом, таблица `products` также соответствует третьей нормальной форме.

3. Transactions (Транзакции):

`transaction_id` - первичный ключ, уникально идентифицирует каждую транзакцию.
`product_id` и `customer_id` - внешние ключи, связывают таблицу с Products и Customers соответственно.
Остальные атрибуты (`transaction_date`, `online_order`, `order_status`) зависят только от `transaction_id`.

Таким образом, таблица Transactions также соответствует третьей нормальной форме, так как неключевые атрибуты зависят только от первичного ключа, и транзитивных зависимостей нет.

## Postgres Screenshots:

![PG Relations](pg_relations.png)
![PG Table](pg_table.png)
![PG Properties](pg_properties.png)